<center>
    
    The Longest Wordle
    
    Author: Daniel Coble
    
    Status: Work in Progress
</center>

In versions 1 and 2 you have seen me fail to find the exact answer to the question of the longest Wordle. After running the depth first search for two days on the largest computer I have access to, it completed 0.007% of the search, but did succeed in finding an 11 word game. In this version, I'm going to give up on doing a full search (and therefore produce a guarantee that it is the longest game). I've developed a 'Solitaire' algorithm which tries to produce long games by developing and assembling long strings of words on the table. There's no guarantee it will produce the best game, but I'll be happy to get a pretty good answer.

Below I'm loading the Wordle lookup table I created in v2. The $M$ boolean matrix will be useful in the future. For the given secret $s$, $M_{ij} $ is whether $j$ would be a valid guess 2 after $i$ was guessed.

In [8]:
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

guess_words = np.loadtxt("guess_words.txt", dtype=str)
secret_words = np.loadtxt("secret_words.txt", dtype=str)
# guess list didn't contain secret_words
guess_words = np.append(guess_words, secret_words)

lookup = np.load("lookup.npy")

secret = 12546 + 500
M = (lookup == lookup[secret].reshape(1, -1)).T

In [9]:
"""
Calculate Wordle for a secret word and guess. The result is encoded as a five element array with 
0 = gray
1 = yellow
2 = green
If asint=True, we take the result array and represent it as a ternary number.

Wordle rules: Greens take precent over yellow. In the case of repeated letters in the guess, the earliest takes precendent.
The same letter will only appear yellow with multiplicity if it appears with multiplicity in the secret word.
"""
def wordle(secret, guess, asint=False):
    # first check greens
    greens = secret == guess
    # fill green spaces with blank to avoid showing as yellow
    secret_ = np.array([' ' if greens[i] else secret[i] for i in range(5)]).astype("|S1")
    # yellow logic
    yellows = np.zeros((5,))
    for i, l in enumerate(guess):
        if((not greens[i]) and (l in secret_)):
            yellows[i] = 1
            secret_[np.where(secret_ == l)[0]] = ' ' # first occurance of letter
    rtrn = (yellows + 2*greens).astype(int)
    if(asint):
        return np.dot(rtrn, np.array([81, 27, 9, 3, 1]))
    return rtrn

"""
Given the guesses and colors, visualize the Wordle game. Guesses and colors must be the same shape.
"""
def visualize(guesses, colors):
    plt.figure(figsize=(5, guesses.shape[0]))
    plt.xlim((-.5, 5))
    plt.ylim((-guesses.shape[0], .5))
    cc = ['lightgray', 'yellow', 'green']
    for y in range(guesses.shape[0]):
        for x in range(5):
            plt.plot([x], [-y], marker='s', c=cc[colors[y, x]], markersize=50)
            plt.text(x-.15, -y-.15, guesses[y, x].astype(str), fontsize=30)
    plt.axis("off")
    plt.tight_layout()
    plt.show()

Here's the rules of the Solitaire algorithm: the table initially contains single hands (ordered lists) ${i}$ for every word *except the secret word*. For every hand and for every word in every hand, we check whether that word and all subsequent words in the list can move to another point in another hand to create either a longer hand. I also had the idea of allowing shifts if it produces a less restricted hand, but I won't implement that here. The algorithm should terminate.

I think the best way to implement this is as a list of linked lists with links pointed downward (to children).

In [10]:
class Node():
    def __init__(self, data, depth=None, child=None, parent=None):
        self.data = data
        self.child = child
        self.parent = parent
        self.depth = depth

'''
function to occasionaly print out information on the table
'''
def printout(table):
    print('%d hands on table'%len(table))
    max_depth = 0
    best_hand = None
    for i, hand in enumerate(table):
        if(hand.depth > max_depth):
            best_hand = hand
            max_depth = hand.depth
    print('best game length %d'%(max_depth+1))
    print('best game: ')
    while(not best_hand is None):
        print(guess_words[best_hand.data])
        best_hand = best_hand.child
    print(guess_words[secret])
    
    
table = [Node(data, 1) for data in range(M.shape[0]) if data != secret]


# now begin the Solitaire algorithm
c = 0
changes = 0 # count how many times hands are moved
iters = 0 # total iterations
while(c != len(table)):
    word1 = table[c] # grab first word in hand
    depth1 = word1.depth
    childmost1_ = word1
    while(not childmost1_.child is None):
        childmost1_ = childmost1_.child
    for d in range(len(table)):
        if(d != c):
            childmost1 = childmost1_
            # count iterations
            iters += 1
            if(iters % 500000 == 0):
                clear_output(wait=True)
                print(str(iters) + ' checks.')
                print(str(changes) + ' swaps.')
                print('c = ' + str(c))
                printout(table)
            word2 = table[d]
            word2child = word2
            depth2 = word2.depth
            add_dep1 = 0
            # find furthest down word2 childmost can be added to
            case = M[word2child.data, childmost1.data]
            while(case):
                add_dep1 += 1
                case = False
                if(not word2child.child is None and M[word2child.child.data, childmost1.data]):
                    word2child = word2child.child
                    case = True
            # find furthest up word1 which can be added
            add_dep2 = 1
            if(add_dep1 > 0):
                case = not childmost1.parent is None
                while(case):
                    parent1 = childmost1.parent
                    wc = word2
                    for i in range(add_dep1):
                        case = case and M[wc.data, parent1.data]
                        wc = wc.child
                    if(case):
                        childmost1 = parent1
                        add_dep2 += 1
                    case = case and not childmost1.parent is None
            # condition to move
            add_dep = add_dep1 + add_dep2
#             if(add_dep == 2):
#                 print(add_dep2)
#                 w2 = word2
#                 print("w2:")
#                 while(not w2 is None):
#                     print(guess_words[w2.data])
#                     w2 = w2.child
#                 print('\n')
#                 cm = childmost1
#                 print('cm:')
#                 while(not cm is None):
#                     print(guess_words[cm.data])
#                     cm = cm.child
#                 print('\n')
            if(add_dep > depth1 and add_dep > depth2):
                if(not childmost1.parent is None):
                    p1 = childmost1.parent
                    p1.child = None
                    # fix depths in word1
                    d = 1
                    while(not p1 is None):
                        p1.depth = d
                        d += 1
                        p1 = p1.parent
                else: # remove the hand from the table
                    table.pop(c)
                if(not word2child.child is None):
                    table.append(word2child.child)
                    word2child.child.parent = None
                childmost1.parent = word2child
                word2child.child = childmost1
                # fix depths in word2
                c2 = childmost1
                while(not c2.child is None):
                    c2 = c2.child
                d = 1
                while(not c2 is None):
                    c2.depth = d
                    d += 1
                    c2 = c2.parent
                # reset
                c = -1
                # count
                changes += 1
                break
    c += 1

62000000 checks.
1199 swaps.
c = 24
13821 hands on table
best game length 4
best game: 
deeds
biffy
ahigh
cumin


KeyboardInterrupt: 

In [ ]:
max_depth = 0
best_hand = None
for i, hand in enumerate(table):
    if(hand.depth > max_depth):
        best_hand = hand
        max_depth = hand.depth
guesses = []
while(not best_hand is None):
        guesses.append(guess_words[best_hand.data])
        best_hand = best_hand.child
guesses.append(guess_words[secret])
guesses = np.array([[l for l in word] for word in guesses]).astype('|S1')
secret_word = np.array([l for l in guess_words[secret]]).astype('|S1')

colors = np.array([wordle(secret_word, guess) for guess in guesses])

visualize(guesses, colors)

In [ ]:
iters